In [ ]:
# Make a Markov aggregate shocks consumer type
AggShockMrkvExample = AggShockMarkovConsumerType(cycles=0)

# Make a Cobb-Douglas economy for the agents
MrkvEconomyExample = CobbDouglasMarkovEconomy(agents=[AggShockMrkvExample])
MrkvEconomyExample.DampingFac = 0.2  # Turn down damping
MrkvEconomyExample.make_AggShkHist()  # Simulate a history of aggregate shocks
MrkvEconomyExample.give_agent_params()  # Have the consumers inherit relevant objects from the economy

In [ ]:
# Solve the microeconomic model for the Markov aggregate shocks example type (and display results)
t_start = process_time()
AggShockMrkvExample.solve()
t_end = process_time()
print(
    "Solving an aggregate shocks Markov consumer took "
    + mystr(t_end - t_start)
    + " seconds.",
)

print(
    "Consumption function at each aggregate market \
        resources-to-labor ratio gridpoint (for each macro state):",
)
m_grid = np.linspace(0, 10, 200)
AggShockMrkvExample.unpack("cFunc")
for i in range(2):
    for M in AggShockMrkvExample.Mgrid.tolist():
        mMin = AggShockMrkvExample.solution[0].mNrmMin[i](M)
        c_at_this_M = AggShockMrkvExample.cFunc[0][i](
            m_grid + mMin,
            M * np.ones_like(m_grid),
        )
        plt.plot(m_grid + mMin, c_at_this_M)
    plt.ylim(0.0, None)
    plt.show()

In [ ]:
# Solve the "macroeconomic" model by searching for a "fixed point dynamic rule"
t_start = process_time()
print("Now solving a two-state Markov economy. This should take a few minutes...")
MrkvEconomyExample.solve()
t_end = process_time()
print(
    'Solving the "macroeconomic" aggregate shocks model took '
    + mystr(t_end - t_start)
    + " seconds.",
)

print(
    "Consumption function at each aggregate market resources-to-labor ratio gridpoint (for each macro state):",
)
m_grid = np.linspace(0, 10, 200)
AggShockMrkvExample.unpack("cFunc")
for i in range(2):
    for M in AggShockMrkvExample.Mgrid.tolist():
        mMin = AggShockMrkvExample.solution[0].mNrmMin[i](M)
        c_at_this_M = AggShockMrkvExample.cFunc[0][i](
            m_grid + mMin,
            M * np.ones_like(m_grid),
        )
        plt.plot(m_grid + mMin, c_at_this_M)
    plt.ylim(0.0, None)
    plt.show()

In [ ]:
# Make a Krusell-Smith agent type
# NOTE: These agents aren't exactly like KS, as they don't have serially correlated unemployment
KSexampleType = AggShockMarkovConsumerType(
    cycles=0,
    PermShkStd=np.array([[0.0, 0.0]]),
    TranShkStd=np.array([[0.0, 0.0]]),
    IncUnemp=np.array([0.0, 0.0]),
    UnempPrb=np.array([0.04, 0.10]),
)

# Make a KS economy
KSeconomy = CobbDouglasMarkovEconomy(
    agents=[KSexampleType], DampingFac=0.2, PermGroFacAgg=[1.0, 1.0]
)
KSeconomy.AggShkDstn = [
    DiscreteDistribution(
        np.array([1.0]),
        np.array([[1.0], [1.05]]),
    ),
    DiscreteDistribution(
        np.array([1.0]),
        np.array([[1.0], [0.95]]),
    ),
]
KSeconomy.give_agent_params()
KSeconomy.make_AggShkHist()

# Solve the K-S model
t_start = process_time()
print(
    "Now solving a Krusell-Smith-style economy. This should take about a minute...",
)
KSeconomy.solve()
t_end = process_time()
print("Solving the Krusell-Smith model took " + mystr(t_end - t_start) + " seconds.")

In [ ]:
# Poly-state example
StateCount = 15  # Number of Markov states
GrowthAvg = 1.01  # Average permanent income growth factor
GrowthWidth = 0.02  # PermGroFacAgg deviates from PermGroFacAgg in this range
Persistence = 0.90  # Probability of staying in the same Markov state
PermGroFacAgg = np.linspace(
    GrowthAvg - GrowthWidth,
    GrowthAvg + GrowthWidth,
    num=StateCount,
)

# Make the Markov array with chosen states and persistence
PolyMrkvArray = np.zeros((StateCount, StateCount))
for i in range(StateCount):
    for j in range(StateCount):
        if i == j:
            PolyMrkvArray[i, j] = Persistence
        elif (i == (j - 1)) or (i == (j + 1)):
            PolyMrkvArray[i, j] = 0.5 * (1.0 - Persistence)
PolyMrkvArray[0, 0] += 0.5 * (1.0 - Persistence)
PolyMrkvArray[StateCount - 1, StateCount - 1] += 0.5 * (1.0 - Persistence)

# Make a consumer type to inhabit the economy
PolyStateExample = AggShockMarkovConsumerType()
PolyStateExample.MrkvArray = PolyMrkvArray
PolyStateExample.PermGroFacAgg = PermGroFacAgg
PolyStateExample.IncShkDstn[0] = StateCount * [PolyStateExample.IncShkDstn[0]]
PolyStateExample.cycles = 0

# Make a Cobb-Douglas economy for the agents
# Use verbose=False to remove printing of intercept
PolyStateEconomy = CobbDouglasMarkovEconomy(
    agents=[PolyStateExample],
    verbose=False,
)
PolyStateEconomy.MrkvArray = PolyMrkvArray
PolyStateEconomy.PermGroFacAgg = PermGroFacAgg
PolyStateEconomy.PermShkAggStd = StateCount * [0.006]
PolyStateEconomy.TranShkAggStd = StateCount * [0.003]
PolyStateEconomy.slope_prev = StateCount * [1.0]
PolyStateEconomy.intercept_prev = StateCount * [0.0]
PolyStateEconomy.update()
PolyStateEconomy.makeAggShkDstn()
PolyStateEconomy.make_AggShkHist()  # Simulate a history of aggregate shocks
PolyStateEconomy.give_agent_params()  # Have the consumers inherit relevant objects from the economy

# Solve the many state model
t_start = process_time()
print(
    "Now solving an economy with "
    + str(StateCount)
    + " Markov states.  This might take a while...",
)
PolyStateEconomy.solve()
t_end = process_time()
print(
    "Solving a model with "
    + str(StateCount)
    + " states took "
    + str(t_end - t_start)
    + " seconds.",
)